In [28]:
!pip install -U sentence-transformers

     |████████████████████████████████| 61kB 1.9MB/s 
     |████████████████████████████████| 573kB 7.4MB/s 
     |████████████████████████████████| 1.0MB 24.7MB/s 
     |████████████████████████████████| 890kB 27.3MB/s 
     |████████████████████████████████| 3.7MB 43.6MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.6.1-cp36-none-any.whl size=74031 sha256=bc93f46ce6ba32b884353a4553aa367e869548f036b8e84d814f636fed676e1a
  Stored in directory: /root/.cache/pip/wheels/d7/fa/17/2b081a8cd8b0a86753fb0e9826b3cc19f0207062c0b2da7008
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=ac1df3db5d5349e6e7139fbd6cce063b44c395b53a1e017313d40e8c6f10c49d
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sentence-transformers sacremoses


In [4]:
### read dataframe after preprocessing
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Deep Learning Data/Covid-19 Text Mining/clean_df.csv', index_col = 0)
df.head()

,paper_id,doi,abstract,body_text,authors,title,journal,abstract_summary,abstract_word_count,body_word_count,body_unique_words,language,processed_text
0,c1a29945f51707d279560855da521ea878989333,10.1126/sciadv.aav4580,Continuously emerging highly pathogenic human ...,Coronaviruses (CoVs) are enveloped viruses wit...,"Xia, Shuai. Yan, Lei. Xu, Wei. Agrawal, An...",A pan-coronavirus fusion inhibitor targeting<...,Sci Adv,Continuously emerging highly pathogenic<br>hu...,150,9082,2521,en,coronaviruses covs envelop virus positive-sens...
1,de4399a1034294c5aecf4f318fd99ca559a762ea,10.7774/cevr.2019.8.1.86,NaN,No potential conflict of interest relevant to ...,"Choi, Jun Yong. Oh, Jin Ok. Ahn, Jin Young....",Absence of neutralizing activity in serum 1<b...,Clin Exp Vaccine Res,Not provided.,0,1060,472,en,potential conflict interest relevant article r...
2,1a2900a53677a6c68ff37167c31ca33ad219acdb,10.4142/jvs.2015.16.4.431,Monoclonal anti-enrofloxacin antibody was prep...,Fluoroquinolones (FQs) have been widely used a...,"Kim, Nam-Gun. Kim, Myeong-Ae. Park, Young-I...",Magnetic nanoparticle based purification and<...,J Vet Sci,Monoclonal anti-enrofloxacin antibody was<br>...,188,2589,1048,en,fluoroquinolones fqs widely use human veterina...
3,61d252869a226c4393e4637cb90fa3b4df082ae3,10.1136/bmjopen-2015-010302,NaN,Fever in the returned traveller is a common sy...,"Jazuli, Farah. Lynd, Terence. Mah, Jordan. ...",Evaluation of a programme for ‘Rapid<br>Asses...,BMJ Open,Not provided.,0,4099,1546,en,fever return traveller common syndrome occur 1...
4,4593d1ecbaa167a2d3e910ccc986f5c4b5ecbbc8,10.1093/epirev/mxz012,The Coalition for Epidemic Preparedness Innova...,"In recent decades, the world has been shaken b...","Gouglas, Dimitrios. Christodoulou, Mario. <b...",CEPI: Driving Progress Toward Epidemic<br>Pre...,Epidemiol Rev,The Coalition for Epidemic Preparedness<br>In...,211,2585,1018,en,recent decade world shake emergence spread new...


In [0]:
### We are gonna use the abstract as summarization of papers, thus we need to further remove the NaN
df = df.dropna()

In [7]:
df.head()

,paper_id,doi,abstract,body_text,authors,title,journal,abstract_summary,abstract_word_count,body_word_count,body_unique_words,language,processed_text
0,c1a29945f51707d279560855da521ea878989333,10.1126/sciadv.aav4580,Continuously emerging highly pathogenic human ...,Coronaviruses (CoVs) are enveloped viruses wit...,"Xia, Shuai. Yan, Lei. Xu, Wei. Agrawal, An...",A pan-coronavirus fusion inhibitor targeting<...,Sci Adv,Continuously emerging highly pathogenic<br>hu...,150,9082,2521,en,coronaviruses covs envelop virus positive-sens...
2,1a2900a53677a6c68ff37167c31ca33ad219acdb,10.4142/jvs.2015.16.4.431,Monoclonal anti-enrofloxacin antibody was prep...,Fluoroquinolones (FQs) have been widely used a...,"Kim, Nam-Gun. Kim, Myeong-Ae. Park, Young-I...",Magnetic nanoparticle based purification and<...,J Vet Sci,Monoclonal anti-enrofloxacin antibody was<br>...,188,2589,1048,en,fluoroquinolones fqs widely use human veterina...
4,4593d1ecbaa167a2d3e910ccc986f5c4b5ecbbc8,10.1093/epirev/mxz012,The Coalition for Epidemic Preparedness Innova...,"In recent decades, the world has been shaken b...","Gouglas, Dimitrios. Christodoulou, Mario. <b...",CEPI: Driving Progress Toward Epidemic<br>Pre...,Epidemiol Rev,The Coalition for Epidemic Preparedness<br>In...,211,2585,1018,en,recent decade world shake emergence spread new...
5,5cba1e45c2829cb0ebaa7bf5384fa685f543c6d6,10.1292/jvms.13-0094,Ascitic feline coronavirus (FCoV) RNA was exam...,Feline infectious peritonitis (FIP) is an immu...,"SOMA, Takehisa. WADA, Makoto. TAHARAGUCHI,<...",Detection of Ascitic Feline Coronavirus RNA<b...,J Vet Med Sci,Ascitic feline coronavirus (FCoV) RNA was<br>...,120,1658,639,en,feline infectious peritonitis fip immune-media...
6,b3707acee35230b2cb6c548837f7f6ca94a26f6b,10.1177/2055116915615370,Case summary A 10-month-old neutered male dome...,Polyuria and polydipsia (PU/PD) are apparent i...,"Long, Charles Tyler. Williams, Morika. <br>S...",Probable primary polydipsia in a domestic<br>...,JFMS Open Rep,Case summary A 10-month-old neutered male<br>...,164,2939,1123,en,polyuria polydipsia pu/pd apparent cat urine s...


In [0]:
import re

df['body_text'] = df['body_text'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))
df['abstract'] = df['abstract'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))

In [0]:
def lower_case(input_str):
    input_str = input_str.lower()
    return input_str

df['body_text'] = df['body_text'].apply(lambda x: lower_case(x))
df['abstract'] = df['abstract'].apply(lambda x: lower_case(x))

In [12]:
df.head(4)

,paper_id,doi,abstract,body_text,authors,title,journal,abstract_summary,abstract_word_count,body_word_count,body_unique_words,language,processed_text
0,c1a29945f51707d279560855da521ea878989333,10.1126/sciadv.aav4580,continuously emerging highly pathogenic human ...,coronaviruses covs are enveloped viruses with ...,"Xia, Shuai. Yan, Lei. Xu, Wei. Agrawal, An...",A pan-coronavirus fusion inhibitor targeting<...,Sci Adv,Continuously emerging highly pathogenic<br>hu...,150,9082,2521,en,coronaviruses covs envelop virus positive-sens...
2,1a2900a53677a6c68ff37167c31ca33ad219acdb,10.4142/jvs.2015.16.4.431,monoclonal antienrofloxacin antibody was prepa...,fluoroquinolones fqs have been widely used as ...,"Kim, Nam-Gun. Kim, Myeong-Ae. Park, Young-I...",Magnetic nanoparticle based purification and<...,J Vet Sci,Monoclonal anti-enrofloxacin antibody was<br>...,188,2589,1048,en,fluoroquinolones fqs widely use human veterina...
4,4593d1ecbaa167a2d3e910ccc986f5c4b5ecbbc8,10.1093/epirev/mxz012,the coalition for epidemic preparedness innova...,in recent decades the world has been shaken by...,"Gouglas, Dimitrios. Christodoulou, Mario. <b...",CEPI: Driving Progress Toward Epidemic<br>Pre...,Epidemiol Rev,The Coalition for Epidemic Preparedness<br>In...,211,2585,1018,en,recent decade world shake emergence spread new...
5,5cba1e45c2829cb0ebaa7bf5384fa685f543c6d6,10.1292/jvms.13-0094,ascitic feline coronavirus fcov rna was examin...,feline infectious peritonitis fip is an immune...,"SOMA, Takehisa. WADA, Makoto. TAHARAGUCHI,<...",Detection of Ascitic Feline Coronavirus RNA<b...,J Vet Med Sci,Ascitic feline coronavirus (FCoV) RNA was<br>...,120,1658,639,en,feline infectious peritonitis fip immune-media...


In [0]:
df.to_csv("/content/drive/My Drive/Deep Learning Data/Covid-19 Text Mining/semantic_covid.csv")

In [15]:
df_covid_test = pd.read_csv("/content/drive/My Drive/Deep Learning Data/Covid-19 Text Mining/semantic_covid.csv")
text = df_covid_test.drop(["authors", "journal", "Unnamed: 0"], axis=1)
text.head(5)

,paper_id,doi,abstract,body_text,title,abstract_summary,abstract_word_count,body_word_count,body_unique_words,language,processed_text
0,c1a29945f51707d279560855da521ea878989333,10.1126/sciadv.aav4580,continuously emerging highly pathogenic human ...,coronaviruses covs are enveloped viruses with ...,A pan-coronavirus fusion inhibitor targeting<...,Continuously emerging highly pathogenic<br>hu...,150,9082,2521,en,coronaviruses covs envelop virus positive-sens...
1,1a2900a53677a6c68ff37167c31ca33ad219acdb,10.4142/jvs.2015.16.4.431,monoclonal antienrofloxacin antibody was prepa...,fluoroquinolones fqs have been widely used as ...,Magnetic nanoparticle based purification and<...,Monoclonal anti-enrofloxacin antibody was<br>...,188,2589,1048,en,fluoroquinolones fqs widely use human veterina...
2,4593d1ecbaa167a2d3e910ccc986f5c4b5ecbbc8,10.1093/epirev/mxz012,the coalition for epidemic preparedness innova...,in recent decades the world has been shaken by...,CEPI: Driving Progress Toward Epidemic<br>Pre...,The Coalition for Epidemic Preparedness<br>In...,211,2585,1018,en,recent decade world shake emergence spread new...
3,5cba1e45c2829cb0ebaa7bf5384fa685f543c6d6,10.1292/jvms.13-0094,ascitic feline coronavirus fcov rna was examin...,feline infectious peritonitis fip is an immune...,Detection of Ascitic Feline Coronavirus RNA<b...,Ascitic feline coronavirus (FCoV) RNA was<br>...,120,1658,639,en,feline infectious peritonitis fip immune-media...
4,b3707acee35230b2cb6c548837f7f6ca94a26f6b,10.1177/2055116915615370,case summary a 10monthold neutered male domest...,polyuria and polydipsia pupd are apparent in a...,Probable primary polydipsia in a domestic<br>...,Case summary A 10-month-old neutered male<br>...,164,2939,1123,en,polyuria polydipsia pu/pd apparent cat urine s...


In [0]:
text_dict = text.to_dict()
len_text = len(text_dict["paper_id"])

In [18]:
from tqdm import tqdm
paper_id_list  = []
body_text_list = []

title_list = []
abstract_list = []
abstract_summary_list = []
for i in tqdm(range(0,len_text)):
  paper_id = text_dict["paper_id"][i]
  body_text = text_dict["body_text"][i].split("\n")
  title = text_dict["title"][i]
  abstract = text_dict["abstract"][i]
  abstract_summary = text_dict["abstract_summary"][i]
  for b in body_text:
    paper_id_list.append(paper_id)
    body_text_list.append(b)
    title_list.append(title)
    abstract_list.append(abstract)
    abstract_summary_list.append(abstract_summary)

100%|██████████| 24239/24239 [00:00<00:00, 25695.95it/s]


In [20]:
df_sentences = pd.DataFrame({"paper_id":paper_id_list},index=body_text_list)
df_sentences.to_csv("/content/drive/My Drive/Deep Learning Data/Covid-19 Text Mining/semantic_sentences.csv")
df_sentences.head()

,paper_id
coronaviruses covs are enveloped viruses with a positivesense singlestranded rna and are associated with various natural hosts covs are divided into alpha beta gamma and delta groups and the beta group is further composed of a b c and d subgroups fig 1a 1 among them six covs can infect humans hcovs including hcov229e 229e and hcovnl63 nl63 in the alpha group hcovoc43 oc43 and hcovhku1 hku1 in beta subgroup a severe acute respiratory syndrome cov sarscov in beta subgroup b and middle east respiratory syndrome cov merscov in beta subgroup c 2,c1a29945f51707d279560855da521ea878989333
in this century sarscov and merscov have emerged in the human population and caused severe pulmonary disease with alarmingly high casefatality rates in 2002 sarscov infections first appeared in china and then quickly spread as a global epidemic in more than 30 countries with 8273 infections and 775 deaths nearly 10 mortality 2 in 2012 merscov emerged in saudi arabia and spread throughout the middle east in 2015 the second pandemic of merscov occurred in south korea causing superspreading events with thirdand fourthgeneration cases of infection the world health organization has reported 2229 laboratoryconfirmed cases of merscov infection including 791 deaths about 35 case fatality in 27 countries as of august 2018 wwwwhointemergencies merscoven meanwhile the remaining common hcovs such as 229e oc43 and nl63 usually infect the human upper respiratory tract and cause the common cold but they also are responsible for severe and even fatal diseases in children elderly and immunocompromised patients 3 4 5 these scenarios suggest that those common hcovs might also pose a lethal threat to humans note that hcovs are rapidly evolving oc43 isolates with novel genomes are being continuously identified 6 7 8,c1a29945f51707d279560855da521ea878989333
the existence of sarslike cov slcov and merslike cov mlcov also pose great threats to public health worldwide recent studies identified some types of slcov such as slwiv1cov and slshc014cov whose spike s proteins highly resemble those of sarscov these slcovs could use the same sarscov receptor ie angiotensinconverting enzyme2 ace2 to directly enter permissive human cells without need for adaptation 9 10 11 in addition the mlcov tylonycteris bat cov hku4 was shown to recognize the merscov receptor cd26 and infect human cells either after introduction of two single mutations s746r and n762a into its s protein or with the help of exogenous protease 12 13 from a historical perspective zoonotic covs harbor a strong ability to cross species barriers to infect humans rapidly and unpredictably as illustrated by newly emerging hcovs 2 14 thus developing a specific drug that only targets a single hcov would be ineffective against newly emerging hcovs 9 10 11 since no broadspectrum antihcov drug is currently available for clinical use it is incumbent to search for a common or conserved target site based on existing hcovs,c1a29945f51707d279560855da521ea878989333
the s glycoprotein is a type i transmembrane glycoprotein that plays an important role in mediating viral infection and is common to all hcovs the s proteins consist of two subunits s1 and s2 fig 1b the s1 subunit binds the cellular receptor through its receptorbinding domain rbd followed by conformational changes in the s2 subunit which allows the fusion peptide to insert into the host target cell membrane the heptad repeat 1 hr1 region in the s2 subunit forms a homotrimeric assembly which exposes three highly conserved hydrophobic grooves on the surface that bind heptad repeat 2 hr2 this sixhelix bundle 6hb core structure is formed during the fusion process and helps bring the viral and cellular membranes into close proximity for viral fusion and entry fig 1b 15 thus the s protein is an important target protein for the development of specific drugs in particular the s1 rbd is a very good target site and both rbdspecific antibodies and rbdbased vaccines have pre

In [22]:
df_sentences = pd.DataFrame({"paper_id":paper_id_list,"title":title_list,"abstract":abstract_list,"abstract_summary":abstract_summary_list},index=body_text_list)
df_sentences.to_csv("/content/drive/My Drive/Deep Learning Data/Covid-19 Text Mining/covid_sentences_Full.csv")
df_sentences.head()

,paper_id,title,abstract,abstract_summary
coronaviruses covs are enveloped viruses with a positivesense singlestranded rna and are associated with various natural hosts covs are divided into alpha beta gamma and delta groups and the beta group is further composed of a b c and d subgroups fig 1a 1 among them six covs can infect humans hcovs including hcov229e 229e and hcovnl63 nl63 in the alpha group hcovoc43 oc43 and hcovhku1 hku1 in beta subgroup a severe acute respiratory syndrome cov sarscov in beta subgroup b and middle east respiratory syndrome cov merscov in beta subgroup c 2,c1a29945f51707d279560855da521ea878989333,A pan-coronavirus fusion inhibitor targeting<...,continuously emerging highly pathogenic human ...,Continuously emerging highly pathogenic<br>hu...
in this century sarscov and merscov have emerged in the human population and caused severe pulmonary disease with alarmingly high casefatality rates in 2002 sarscov infections first appeared in china and then quickly spread as a global epidemic in more than 30 countries with 8273 infections and 775 deaths nearly 10 mortality 2 in 2012 merscov emerged in saudi arabia and spread throughout the middle east in 2015 the second pandemic of merscov occurred in south korea causing superspreading events with thirdand fourthgeneration cases of infection the world health organization has reported 2229 laboratoryconfirmed cases of merscov infection including 791 deaths about 35 case fatality in 27 countries as of august 2018 wwwwhointemergencies merscoven meanwhile the remaining common hcovs such as 229e oc43 and nl63 usually infect the human upper respiratory tract and cause the common cold but they also are responsible for severe and even fatal diseases in children elderly and immunocompromised patients 3 4 5 these scenarios suggest that those common hcovs might also pose a lethal threat to humans note that hcovs are rapidly evolving oc43 isolates with novel genomes are being continuously identified 6 7 8,c1a29945f51707d279560855da521ea878989333,A pan-coronavirus fusion inhibitor targeting<...,continuously emerging highly pathogenic human ...,Continuously emerging highly pathogenic<br>hu...
the existence of sarslike cov slcov and merslike cov mlcov also pose great threats to public health worldwide recent studies identified some types of slcov such as slwiv1cov and slshc014cov whose spike s proteins highly resemble those of sarscov these slcovs could use the same sarscov receptor ie angiotensinconverting enzyme2 ace2 to directly enter permissive human cells without need for adaptation 9 10 11 in addition the mlcov tylonycteris bat cov hku4 was shown to recognize the merscov receptor cd26 and infect human cells either after introduction of two single mutations s746r and n762a into its s protein or with the help of exogenous protease 12 13 from a historical perspective zoonotic covs harbor a strong ability to cross species barriers to infect humans rapidly and unpredictably as illustrated by newly emerging hcovs 2 14 thus developing a specific drug that only targets a single hcov would be ineffective against newly emerging hcovs 9 10 11 since no broadspectrum antihcov drug is currently available for clinical use it is incumbent to search for a common or conserved target site based on existing hcovs,c1a29945f51707d279560855da521ea878989333,A pan-coronavirus fusion inhibitor targeting<...,continuously emerging highly pathogenic human ...,Continuously emerging highly pathogenic<br>hu...
the s glycoprotein is a type i transmembrane glycoprotein that plays an important role in mediating viral infection and is common to all hcovs the s proteins consist of two subunits s1 and s2 fig 1b the s1 subunit binds the cellular receptor through its receptorbinding domain rbd followed by conformational changes in the s2 subunit which allows the fusion peptide to insert into the host target cell membrane the heptad repeat 1 hr1 region in the s2 subunit forms a homotrimeric assembly which expos

In [23]:
df_sentences = df_sentences["paper_id"].to_dict()
df_sentences_list = list(df_sentences.keys())
len(df_sentences_list)

775361

In [24]:
list(df_sentences.keys())[:5]

['coronaviruses covs are enveloped viruses with a positivesense singlestranded rna and are associated with various natural hosts covs are divided into alpha beta gamma and delta groups and the beta group is further composed of a b c and d subgroups fig 1a 1 among them six covs can infect humans hcovs including hcov229e 229e and hcovnl63 nl63 in the alpha group hcovoc43 oc43 and hcovhku1 hku1 in beta subgroup a severe acute respiratory syndrome cov sarscov in beta subgroup b and middle east respiratory syndrome cov merscov in beta subgroup c 2 ',
 'in this century sarscov and merscov have emerged in the human population and caused severe pulmonary disease with alarmingly high casefatality rates in 2002 sarscov infections first appeared in china and then quickly spread as a global epidemic in more than 30 countries with 8273 infections and 775 deaths nearly 10 mortality 2  in 2012 merscov emerged in saudi arabia and spread throughout the middle east in 2015 the second pandemic of merscov

In [25]:
df_sentences_list = [str(d) for d in tqdm(df_sentences_list)]

100%|██████████| 775361/775361 [00:00<00:00, 1918020.55it/s]


In [26]:
import pandas as pd
df = pd.read_csv("/content/drive/My Drive/Deep Learning Data/Covid-19 Text Mining/covid_sentences_Full.csv", index_col=0)
df.head()

,paper_id,title,abstract,abstract_summary
coronaviruses covs are enveloped viruses with a positivesense singlestranded rna and are associated with various natural hosts covs are divided into alpha beta gamma and delta groups and the beta group is further composed of a b c and d subgroups fig 1a 1 among them six covs can infect humans hcovs including hcov229e 229e and hcovnl63 nl63 in the alpha group hcovoc43 oc43 and hcovhku1 hku1 in beta subgroup a severe acute respiratory syndrome cov sarscov in beta subgroup b and middle east respiratory syndrome cov merscov in beta subgroup c 2,c1a29945f51707d279560855da521ea878989333,A pan-coronavirus fusion inhibitor targeting<...,continuously emerging highly pathogenic human ...,Continuously emerging highly pathogenic<br>hu...
in this century sarscov and merscov have emerged in the human population and caused severe pulmonary disease with alarmingly high casefatality rates in 2002 sarscov infections first appeared in china and then quickly spread as a global epidemic in more than 30 countries with 8273 infections and 775 deaths nearly 10 mortality 2 in 2012 merscov emerged in saudi arabia and spread throughout the middle east in 2015 the second pandemic of merscov occurred in south korea causing superspreading events with thirdand fourthgeneration cases of infection the world health organization has reported 2229 laboratoryconfirmed cases of merscov infection including 791 deaths about 35 case fatality in 27 countries as of august 2018 wwwwhointemergencies merscoven meanwhile the remaining common hcovs such as 229e oc43 and nl63 usually infect the human upper respiratory tract and cause the common cold but they also are responsible for severe and even fatal diseases in children elderly and immunocompromised patients 3 4 5 these scenarios suggest that those common hcovs might also pose a lethal threat to humans note that hcovs are rapidly evolving oc43 isolates with novel genomes are being continuously identified 6 7 8,c1a29945f51707d279560855da521ea878989333,A pan-coronavirus fusion inhibitor targeting<...,continuously emerging highly pathogenic human ...,Continuously emerging highly pathogenic<br>hu...
the existence of sarslike cov slcov and merslike cov mlcov also pose great threats to public health worldwide recent studies identified some types of slcov such as slwiv1cov and slshc014cov whose spike s proteins highly resemble those of sarscov these slcovs could use the same sarscov receptor ie angiotensinconverting enzyme2 ace2 to directly enter permissive human cells without need for adaptation 9 10 11 in addition the mlcov tylonycteris bat cov hku4 was shown to recognize the merscov receptor cd26 and infect human cells either after introduction of two single mutations s746r and n762a into its s protein or with the help of exogenous protease 12 13 from a historical perspective zoonotic covs harbor a strong ability to cross species barriers to infect humans rapidly and unpredictably as illustrated by newly emerging hcovs 2 14 thus developing a specific drug that only targets a single hcov would be ineffective against newly emerging hcovs 9 10 11 since no broadspectrum antihcov drug is currently available for clinical use it is incumbent to search for a common or conserved target site based on existing hcovs,c1a29945f51707d279560855da521ea878989333,A pan-coronavirus fusion inhibitor targeting<...,continuously emerging highly pathogenic human ...,Continuously emerging highly pathogenic<br>hu...
the s glycoprotein is a type i transmembrane glycoprotein that plays an important role in mediating viral infection and is common to all hcovs the s proteins consist of two subunits s1 and s2 fig 1b the s1 subunit binds the cellular receptor through its receptorbinding domain rbd followed by conformational changes in the s2 subunit which allows the fusion peptide to insert into the host target cell membrane the heptad repeat 1 hr1 region in the s2 subunit forms a homotrimeric assembly which expos

In [32]:
#https://github.com/UKPLab/sentence-transformers/blob/master/examples/application_semantic_search.py
"""
This is a simple application for sentence embeddings: semantic search
We have a corpus with various sentences. Then, for a given query sentence,
we want to find the most similar sentence in this corpus.
This script outputs for various queries the top 5 most similar sentences in the corpus.
"""

from sentence_transformers import SentenceTransformer
import scipy.spatial
import pickle as pkl
embedder = SentenceTransformer('bert-base-nli-mean-tokens')

# Corpus with example sentences
corpus = df_sentences_list
corpus_embeddings = embedder.encode(corpus,show_progress_bar=True)
with open("/content/drive/My Drive/BertSentenceSimilarity/Pickles/corpus_embeddings.pkl" , "rb") as file_:
  corpus_embeddings = pkl.load(file_)

# Query sentences:
queries = ['What has been published about medical care?',
           'Knowledge of the frequency, manifestations, and course of extrapulmonary manifestations of COVID-19, including, but not limited to, possible cardiomyopathy and cardiac arrest',
           'Use of AI in real-time health care delivery to evaluate interventions, risk factors, and outcomes in a way that could not be done manually',
           'Resources to support skilled nursing facilities and long term care facilities.',
           'Mobilization of surge medical staff to address shortages in overwhelmed communities .',
           'Age-adjusted mortality data for Acute Respiratory Distress Syndrome (ARDS) with/without other organ failure – particularly for viral etiologies .']
query_embeddings = embedder.encode(queries,show_progress_bar=True)

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
closest_n = 5
print("\nTop 5 most similar sentences in corpus:")
for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n=========================================================")
    print("==========================Query==============================")
    print("===",query,"=====")
    print("=========================================================")


    for idx, distance in results[0:closest_n]:
        print("Score:   ", "(Score: %.4f)" % (1-distance) , "\n" )
        print("Paragraph:   ", corpus[idx].strip(), "\n" )
        row_dict = df.loc[df.index== corpus[idx]].to_dict()
        print("paper_id:  " , row_dict["paper_id"][corpus[idx]] , "\n")
        print("Title:  " , row_dict["title"][corpus[idx]] , "\n")
        print("Abstract:  " , row_dict["abstract"][corpus[idx]] , "\n")
        print("Abstract_Summary:  " , row_dict["abstract_summary"][corpus[idx]] , "\n")
        print("-------------------------------------------")

KeyboardInterrupt: ignored